<a href="https://colab.research.google.com/github/ThomasLane1820/Prediction-of-Product-Sales/blob/main/Production_of_Product_Sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prediction of Product Sales
- Author: Thomas Lane


## Project overview

## Load and Inspect Data

### Loading Data

In [35]:
# Mount google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:
# Importing required packages
import pandas as pd
fpath = '/content/drive/MyDrive/Coding_dojo_work/StudentFolders16wk24wk/CodingDojo/01-Fundamentals/Week02/Data/sales_predictions_2023.csv'
df = pd.read_csv(fpath)

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [38]:
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [39]:
df.describe()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,7060.000000,8523.000000,8523.000000,8523.000000,8523.000000
mean,12.857645,0.066132,140.992782,1997.831867,2181.288914
std,4.643456,0.051598,62.275067,8.371760,1706.499616
min,4.555000,0.000000,31.290000,1985.000000,33.290000
25%,8.773750,0.026989,93.826500,1987.000000,834.247400
50%,12.600000,0.053931,143.012800,1999.000000,1794.331000
75%,16.850000,0.094585,185.643700,2004.000000,3101.296400
max,21.350000,0.328391,266.888400,2009.000000,13086.964800


## Clean Data

### Data Cleaning

- There are 8,523 rows, and 12 columns. There are 7 categorical/string columns and 5 numerical columns.

#### Duplicates

In [40]:
# Creating a filter that identifies duplicated rows.
duplicated_rows = df.duplicated()
# finding the sum of duplicated rows.
duplicated_rows.sum()

0

- There are no duplicated rows.
- Now it's time to check and see if there are any missing values.

#### Missing Values

In [41]:
# getting the number of null values in each column and saving it as a variable.
null_values = df.isna().sum()
null_values

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

- The only columns that have nulls are "Item_weight" abd "Outlet_Size", and they each have over 1000 nulls. I'll address this by using placeholder values.

In [42]:
# Replacing the missing "Item_Weight" values with the placeholder value of (-1).
df["Item_Weight"] = df["Item_Weight"].fillna(-1)

In [43]:
# Replacing the missing "Outlet_Size" values with the placeholder value of "MISSING".
df['Outlet_Size'] = df["Outlet_Size"].fillna("MISSING")

In [44]:
# Verifying that all missing values were properly replaced.
null_values = df.isna().sum()
null_values

Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
Item_Outlet_Sales            0
dtype: int64

- There are no more missing null values in our data set.
- Now to cehck and see if there are any inconsistent data categories.

#### Data Inconsistencies

##### Categorical inconsistencies

In [45]:
# Saving a list of object columns
string_cols = df.select_dtypes('object').columns
string_cols

Index(['Item_Identifier', 'Item_Fat_Content', 'Item_Type', 'Outlet_Identifier',
       'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type'],
      dtype='object')

In [46]:
# Checking the number of unique values in each object column.
df[string_cols].nunique()

Item_Identifier         1559
Item_Fat_Content           5
Item_Type                 16
Outlet_Identifier         10
Outlet_Size                4
Outlet_Location_Type       3
Outlet_Type                4
dtype: int64

In [47]:
df[string_cols].nunique() / len(df) * 100

Item_Identifier         18.291681
Item_Fat_Content         0.058665
Item_Type                0.187727
Outlet_Identifier        0.117330
Outlet_Size              0.046932
Outlet_Location_Type     0.035199
Outlet_Type              0.046932
dtype: float64

In [48]:

# loop through the list of string columns
for col in string_cols:
  # print the value counts for the column
  print(f'Value counts for {col}')
  print(df[col].value_counts())
  # Printing an empty line for readability
  print('\n')

Value counts for Item_Identifier
FDW13    10
FDG33    10
NCY18     9
FDD38     9
DRE49     9
         ..
FDY43     1
FDQ60     1
FDO33     1
DRF48     1
FDC23     1
Name: Item_Identifier, Length: 1559, dtype: int64


Value counts for Item_Fat_Content
Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64


Value counts for Item_Type
Fruits and Vegetables    1232
Snack Foods              1200
Household                 910
Frozen Foods              856
Dairy                     682
Canned                    649
Baking Goods              648
Health and Hygiene        520
Soft Drinks               445
Meat                      425
Breads                    251
Hard Drinks               214
Others                    169
Starchy Foods             148
Breakfast                 110
Seafood                    64
Name: Item_Type, dtype: int64


Value counts for Outlet_Identifier
OUT027    935
OUT013    932
OUT049    930
OUT046    930


- Using this method we can see a there are a few inconsistencies in Item_Fat_Content. Where Low fat, LF, low fat, reg, Regular can be simplified
- Along with weird phrasing in Outlet_Size where it goes from Small, Medium, and High.

In [50]:
# Replacing "LF" and "low fat" with "Low Fat" in Item_Fat_Content.
df['Item_Fat_Content'] = df['Item_Fat_Content'].replace({'LF': 'Low Fat',
                                                         'low fat': 'Low Fat'})
df['Item_Fat_Content'].value_counts()

Low Fat    5517
Regular    2889
reg         117
Name: Item_Fat_Content, dtype: int64

In [51]:
# Replacing "reg" with "Regular".
df['Item_Fat_Content'] = df['Item_Fat_Content'].replace({'reg': 'Regular'})
# Ensuring that the oddities are resovled
df['Item_Fat_Content'].value_counts()

Low Fat    5517
Regular    3006
Name: Item_Fat_Content, dtype: int64

In [52]:
# Replacing "High" with "Large" in Outlet_Size for sizing terminology consistency
df['Outlet_Size'] = df['Outlet_Size'].replace({'High': 'Large'})
# Ensuring that the oddities are resovled
df['Outlet_Size'].value_counts()

Medium     2793
MISSING    2410
Small      2388
Large       932
Name: Outlet_Size, dtype: int64

- Now all inconsistencies in Item_Fat_Content are resolved.
- While there were no clear oddities in Outlet_Size and the "fix" was largely preferential, that column is also resolved.
- Now it's time to check the numerical inconsistencies.

- The "Item_Identifier" Has a lot of repeat values, I want to verify that these are intended.

In [49]:
# Verifying that the duplicate "Item_Identifier" values are unique rather than near copies
df[df['Item_Identifier'] == 'FDW13']

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
256,FDW13,8.5,Low Fat,0.098438,Canned,51.1324,OUT017,2007,MISSING,Tier 2,Supermarket Type1,259.6620
1141,FDW13,8.5,Low Fat,0.097866,Canned,51.1324,OUT035,2004,Small,Tier 2,Supermarket Type1,1350.2424
2190,FDW13,8.5,Low Fat,0.098283,Canned,51.3324,OUT018,2009,Medium,Tier 3,Supermarket Type2,934.7832
3928,FDW13,-1.0,Low Fat,0.097411,Canned,52.7324,OUT027,1985,Medium,Tier 3,Supermarket Type3,2492.7552
3978,FDW13,8.5,Low Fat,0.098037,Canned,51.5324,OUT049,1999,Medium,Tier 1,Supermarket Type1,882.8508
6499,FDW13,8.5,Low Fat,0.098083,Canned,50.3324,OUT045,2002,MISSING,Tier 2,Supermarket Type1,675.1212
6587,FDW13,8.5,low fat,0.163839,Canned,51.3324,OUT010,1998,MISSING,Tier 3,Grocery Store,311.5944
7029,FDW13,8.5,Low Fat,0.097803,Canned,50.5324,OUT013,1987,High,Tier 3,Supermarket Type1,882.8508
8025,FDW13,-1.0,Low Fat,0.171384,Canned,50.5324,OUT019,1985,Small,Tier 1,Grocery Store,207.7296
8032,FDW13,8.5,Low Fat,0.097885,Canned,52.5324,OUT046,1997,Small,Tier 1,Supermarket Type1,1194.4452


##### Numerical Inconsistencies

In [53]:
# Saving a list of numerical columns
num_cols = df.select_dtypes('number').columns
num_cols

Index(['Item_Weight', 'Item_Visibility', 'Item_MRP',
       'Outlet_Establishment_Year', 'Item_Outlet_Sales'],
      dtype='object')

In [54]:
# Finding the mean along with max and min values
for col in num_cols:
  # print the description of each column
  print(f'Value counts for {col}')
  print(df[col].describe())
  # Print an empty line for readability
  print('\n')


Value counts for Item_Weight
count    8523.000000
mean       10.478936
std         6.720742
min        -1.000000
25%         6.650000
50%        11.000000
75%        16.000000
max        21.350000
Name: Item_Weight, dtype: float64


Value counts for Item_Visibility
count    8523.000000
mean        0.066132
std         0.051598
min         0.000000
25%         0.026989
50%         0.053931
75%         0.094585
max         0.328391
Name: Item_Visibility, dtype: float64


Value counts for Item_MRP
count    8523.000000
mean      140.992782
std        62.275067
min        31.290000
25%        93.826500
50%       143.012800
75%       185.643700
max       266.888400
Name: Item_MRP, dtype: float64


Value counts for Outlet_Establishment_Year
count    8523.000000
mean     1997.831867
std         8.371760
min      1985.000000
25%      1987.000000
50%      1999.000000
75%      2004.000000
max      2009.000000
Name: Outlet_Establishment_Year, dtype: float64


Value counts for Item_Outlet_Sales
cou

- Upon first inspection the numerical columns seems to not have any apparent inconsistencies or oddities, other than Item_Weight which we replaced the missing values with -1.
- If we look back at the "Loading Data" section we find that the min for Item_Weight was 4.555.

## Exploratory Data analysis

## Feature inspection